In [ ]:
import tensorflow as tf
import os
import shutil
import pandas as pd
import re
from bs4 import BeautifulSoup
import seaborn as sns
import numpy as np

import os
import numpy as np
import pandas as pd
from google.colab import drive

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/database/movie_variable_full.csv")
df[df.isnull().T.any()]
df.head(10)

,id,tconst,titleType,primaryTitle,originalTitle,startYear,release_date,isAdult,genres,runtime,...,production_companies,production_countries,revenue,status,vote_average,vote_count,averageRating,numVotes,director,actor
0,16612,tt0000001,short,Carmencita,Carmencita,1894.0,1894-03-14,0,"Documentary,Short",1.0,...,Edison Manufacturing Company,US,0.0,Released,4.9,18.0,5.6,1550.0,William K.L. Dickson,Carmencita
1,88013,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,1892.0,1892-10-28,0,"Animation,Comedy,Romance",4.0,...,None,FR,0.0,Released,6.1,19.0,6.5,1207.0,Émile Reynaud,None
2,16624,tt0000005,short,Blacksmith Scene,Blacksmith Scene,1893.0,1893-05-08,0,"Comedy,Short",1.0,...,Edison Manufacturing Company,US,0.0,Released,5.8,19.0,6.1,1934.0,William K.L. Dickson,Charles Kayser
3,105158,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,1894.0,1894-01-09,0,"Documentary,Short",1.0,...,Edison Manufacturing Company,US,0.0,Released,4.7,12.0,5.4,1667.0,William K.L. Dickson,Fred Ott
4,774,tt0000010,short,Exiting the Factory,La sortie de l'usine Lumière à Lyon,1895.0,1895-06-10,0,"Documentary,Short",1.0,...,Société Lumière,FR,0.0,Released,6.2,52.0,6.9,5545.0,Louis Lumière,None
5,160,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,1896.0,1896-01-25,0,"Action,Documentary,Short",1.0,...,Lumière,FR,0.0,Released,6.9,87.0,7.4,9435.0,Auguste Lumière,Jeanne-Joséphine Lumière
6,82120,tt0000014,short,The Sprinkler Sprinkled,L'arroseur arrosé,1895.0,1895-12-27,0,"Comedy,Short",1.0,...,Lumière,FR,0.0,Released,7.0,44.0,7.1,4111.0,Louis Lumière,François Clerc
7,127762,tt0000023,short,The Sea,Baignade en mer,1895.0,1895-12-28,0,"Documentary,Short",1.0,...,Lumière,FR,0.0,Released,5.3,12.0,5.6,1028.0,Louis Lumière,None
8,122134,tt0000029,short,Baby's Dinner,Repas de bébé,1895.0,1895-12-27,0,"Documentary,Short",1.0,...,Société Lumière,FR,0.0,Released,5.4,22.0,5.9,2639.0,Louis Lumière,Auguste Lumière
9,129865,tt0000070,short,Démolition d'un mur,Démolition d'un mur,1896.0,1896-03-06,0,"Documentary,Short",1.0,...,Lumière,FR,0.0,Released,5.8,17.0,6.4,2087.0,Louis Lumière,Auguste Lumière


In [ ]:
# df = df.dropna()
df.columns

Index(['id', 'tconst', 'titleType', 'primaryTitle', 'originalTitle',
       'startYear', 'release_date', 'isAdult', 'genres', 'runtime', 'budget',
       'original_language', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'revenue', 'status',
       'vote_average', 'vote_count', 'averageRating', 'numVotes', 'director',
       'actor'],
      dtype='object')

In [ ]:
df.shape

(45307, 25)

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 37.0 MB/s 
     |████████████████████████████████| 596 kB 37.8 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
# Load pre-trained model tokenizer (vocabulary)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# Load pre-trained model (weights)
fixedbert = DistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
fixedbert

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [ ]:
df_b = df[['overview','averageRating']]
# labels=df_b['Outcome']
df_b

,overview,averageRating
0,The first woman to appear in front of an Ediso...,5.6
1,"One night, Arlequin come to see his lover Colo...",6.5
2,Three men hammer on an anvil and pass a bottle...,6.1
3,A man (Thomas Edison's assistant) takes a pinc...,5.4
4,Working men and women leave the Lumière factor...,6.9
...,...,...
45302,Unbridled comic Chris D'Elia reconsiders his a...,6.5
45303,"Stranded in an Arctic mine, two survivors are ...",7.3
45304,"Set during the Vietnam war, Firebase follows A...",7.2
45305,"William Shatner sits down with scientists, inn...",6.7


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!/opt/bin/nvidia-smi

Fri Apr 15 12:20:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    62W / 149W |    145MiB / 11441MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
device=torch.device('cuda:0')
fixedbert=fixedbert.to(device)

In [ ]:
tokenized = df_b['overview'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(45307, 512)

In [ ]:
feature_list = []
with torch.no_grad():
    for batch_idx in range(0,padded.shape[0],20):
        #BERT check 20 sample each time.
        input_ids = torch.tensor(padded[batch_idx:batch_idx+20]).to(device)
        used_attention_mask = torch.tensor(attention_mask[batch_idx:batch_idx+20]).to(device)
        last_hidden_states=fixedbert(input_ids,attention_mask=used_attention_mask)
        #Get the embeddings for the [CLS] tag (position is 0)
        features = last_hidden_states[0][:,0,:].cpu().numpy()
        feature_list.append(features)
        print(batch_idx)
        print(len(feature_list))

0
1
20
2
40
3
60
4
80
5
100
6
120
7
140
8
160
9
180
10
200
11
220
12
240
13
260
14
280
15
300
16
320
17
340
18
360
19
380
20
400
21
420
22
440
23
460
24
480
25
500
26
520
27
540
28
560
29
580
30
600
31
620
32
640
33
660
34
680
35
700
36
720
37
740
38
760
39
780
40
800
41
820
42
840
43
860
44
880
45
900
46
920
47
940
48
960
49
980
50
1000
51
1020
52
1040
53
1060
54
1080
55
1100
56
1120
57
1140
58
1160
59
1180
60
1200
61
1220
62
1240
63
1260
64
1280
65
1300
66
1320
67
1340
68
1360
69
1380
70
1400
71
1420
72
1440
73
1460
74
1480
75
1500
76
1520
77
1540
78
1560
79
1580
80
1600
81
1620
82
1640
83
1660
84
1680
85
1700
86
1720
87
1740
88
1760
89
1780
90
1800
91
1820
92
1840
93
1860
94
1880
95
1900
96
1920
97
1940
98
1960
99
1980
100
2000
101
2020
102
2040
103
2060
104
2080
105
2100
106
2120
107
2140
108
2160
109
2180
110
2200
111
2220
112
2240
113
2260
114
2280
115
2300
116
2320
117
2340
118
2360
119
2380
120
2400
121
2420
122
2440
123
2460
124
2480
125
2500
126
2520
127
2540
128
2560
129
258

In [ ]:
len(feature_list)

2266

In [ ]:
# preprare features
features = np.vstack(feature_list)
features.shape

(45307, 768)

In [ ]:
df_a = df[['id','averageRating']]
# df_a = df_a.reset_index()
# df_a.drop(columns={'index'})
df_a

,id,averageRating
0,16612,5.6
1,88013,6.5
2,16624,6.1
3,105158,5.4
4,774,6.9
...,...,...
45302,462108,6.5
45303,464111,7.3
45304,463800,7.2
45305,464207,6.7


In [ ]:
dfft=pd.DataFrame(features)
dfft=pd.concat([dfft,df_a],axis=1)
dfft



,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,id,averageRating
0,-0.212757,-0.119363,-0.467321,-0.104196,0.156821,0.016843,0.453776,0.254775,-0.587311,-0.117029,...,-0.012145,-0.167214,-0.138516,0.032850,-0.055581,-0.188506,0.389635,0.614132,16612,5.6
1,-0.529347,-0.111475,0.185557,-0.299205,0.231830,0.061060,0.577831,0.435808,-0.434813,-0.551514,...,0.127186,-0.225243,-0.001103,0.209374,-0.023503,-0.042717,0.279547,0.288125,88013,6.5
2,-0.075160,-0.204142,-0.309244,-0.201729,0.050463,-0.129172,0.451516,0.181387,-0.181858,-0.122962,...,-0.076439,-0.047331,-0.064747,0.434560,0.040172,-0.140817,0.358042,0.289663,16624,6.1
3,-0.150102,-0.030070,-0.154968,-0.020246,0.183364,-0.209394,0.328196,0.328270,-0.419871,-0.214888,...,-0.139995,0.014957,0.140709,0.249312,-0.108817,-0.080509,0.326759,0.749857,105158,5.4
4,-0.127788,-0.165061,-0.116118,-0.133790,0.093125,0.015205,0.448658,0.174943,-0.514834,-0.324614,...,-0.046860,-0.117528,-0.057998,0.141944,0.031960,-0.107709,0.399279,0.567217,774,6.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45302,-0.352658,-0.285928,-0.344132,-0.184588,-0.083109,-0.053796,0.296736,0.397172,0.109560,-0.346092,...,0.172909,-0.107937,0.153141,-0.053051,0.001906,-0.064014,0.481054,0.121861,462108,6.5
45303,-0.155336,-0.085507,-0.213127,-0.140744,0.063877,-0.077810,0.531019,0.349855,0.083759,-0.465800,...,-0.033241,-0.302728,0.104000,0.228341,-0.164201,-0.178058,0.419185,0.410078,464111,7.3
45304,-0.257478,-0.264399,-0.232159,-0.229921,0.093284,-0.073401,0.298765,0.297948,0.430782,-0.259244,...,-0.147157,-0.571072,-0.006006,0.065522,-0.089352,0.126458,0.560034,0.348056,463800,7.2
45305,-0.085204,-0.102886,-0.205029,-0.145162,-0.022423,-0.174823,0.246258,0.443171,0.011585,-0.265844,...,0.160330,-0.316926,-0.008686,-0.005832,-0.009572,-0.227925,0.692969,0.199666,464207,6.7


In [ ]:
dfft.to_csv('/content/drive/MyDrive/database/5153_overview_bert.csv')

In [ ]:
dfft = dfft.drop(columns={'id','averageRating'})

In [ ]:
#get labels
from sklearn.model_selection import train_test_split
labels = df.averageRating.tolist()
for i in range(len(labels)):
  if labels[i]>=6.5:
    labels[i]=1
  else:
    labels[i]=0
train_features, test_features, train_labels, test_labels = train_test_split(dfft, labels, test_size=0.25) #random_state=5)

In [ ]:
labels

[0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
lr_clf = LogisticRegression(max_iter=20000)
lr_clf.fit(train_features, train_labels)

#validate the model
lr_clf.score(test_features, test_labels)

0.6787322327182838